In [25]:
import os
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
# from imutils import paths

import matplotlib.pyplot as plt
import ast
from pathlib import Path

In [141]:
print(tf.__version__)

2.10.1


In [145]:
DATA_PATH = Path('.')

# Load files,r'' C:/Users/good/Documents/ziki/hackatons/vhidya
# Define image and label paths
image_dir = 'C:/Users/good/Documents/ziki/hackatons/vhidya/garments_vision/data/train_LbELtWX/train'  # Update with the actual image folder path

test_image_dir = 'C:/Users/good/Documents/ziki/hackatons/vhidya/garments_vision/data/test_ScVgIM0/test'

# Load files
data = pd.read_csv(DATA_PATH / 'data/train_LbELtWX/train.csv', dtype=str)
test = pd.read_csv(DATA_PATH / 'data/test_ScVgIM0/test.csv', dtype=str)
ss = pd.read_csv(DATA_PATH / 'data/sample_submission_I5njJSF.csv')

data['id']=data['id'].apply(lambda x: f'{x}.png')
test['id']=test['id'].apply(lambda x: f'{x}.png')

# Ensure that classes are integers
data['label'] = data['label'].astype(str)

In [146]:
imgs = os.listdir(image_dir)
print(imgs[:10])

['1.png', '10.png', '100.png', '1000.png', '10000.png', '10001.png', '10002.png', '10003.png', '10004.png', '10005.png']


In [147]:
# Create ImageDataGenerator instances
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2, #Use 20% of training data for validation
    horizontal_flip=True,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1
    )

In [151]:
test_datagen = ImageDataGenerator(rescale=1./255)

#Create generators
train_generator = train_datagen.flow_from_dataframe(
    data,
    directory = image_dir,
    x_col='id',
    y_col='label',
    has_extension = True,  # save_format = 'png',
    target_size=(128, 128), #You can adjust the size as needed
    batch_size=32,
    class_mode='categorical',
    classes=["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"],
    subset='training',
    shuffle=True
    )


Found 48000 validated image filenames belonging to 10 classes.


In [152]:
validation_generator=train_datagen.flow_from_dataframe(
    data,
    directory=image_dir,
    x_col='id',
    y_col='label',
    target_size=(128,128),
    batch_size=32,
    save_format = 'png',
    class_mode='categorical',
    subset='validation',
    classes=["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"],
    shuffle=True
    )



Found 12000 validated image filenames belonging to 10 classes.


In [153]:


test_datagen=ImageDataGenerator(rescale=1./255)


test_generator=test_datagen.flow_from_dataframe(
    test_df,
    directory = test_image_dir,
    x_col='id',
    y_col='label',
    target_size=(128,128),
    batch_size=32,
    class_mode= None,
    shuffle=False
    )

Found 10000 validated image filenames.


In [154]:
def build_model(input_shape,num_classes):
    model = Sequential([
        Conv2D(32,(3,3), activation='relu',input_shape=input_shape),
        MaxPooling2D((2,2)),
        Conv2D(64,(3,3), activation='relu'),
        MaxPooling2D((2,2)),
        Conv2D(128, (3,3), activation='relu'),
        MaxPooling2D((2,2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    
    model.compile(optimizer=Adam(learning_rate=0.001),loss='categorical_crossentropy', metrics=['accuracy'])
    return model

#Image size (128x128) and 3 channels(RGB)
input_shape = (128, 128, 3) 

num_classes = len(train_generator.class_indices) #Number of classes
model = build_model(input_shape, num_classes)


In [155]:

#Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

model_checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_loss', mode='min')

In [157]:
#Train the model
history = model.fit(
    train_generator,
    epochs= 2,               # 50,
    validation_data = validation_generator,
    callbacks = [early_stopping, model_checkpoint]
    )

#Load the best model
model.load_weights('best_model.h5')

#Evaluate on the test set
test_loss, test_acc = model.evaluate(test_generator)

print(f'TestAccuracy:{test_acc:.2f}')

Epoch 1/2
1044/1500 [===================>..........] - ETA: 21:20 - loss: 0.6703 - accuracy: 0.7482

UnknownError: Graph execution error:

UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x000002C463A5E040>
Traceback (most recent call last):

  File "D:\Programs\Anaconda\lib\site-packages\tensorflow\python\ops\script_ops.py", line 271, in __call__
    ret = func(*args)

  File "D:\Programs\Anaconda\lib\site-packages\tensorflow\python\autograph\impl\api.py", line 642, in wrapper
    return func(*args, **kwargs)

  File "D:\Programs\Anaconda\lib\site-packages\tensorflow\python\data\ops\dataset_ops.py", line 1035, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "D:\Programs\Anaconda\lib\site-packages\keras\engine\data_adapter.py", line 903, in wrapped_generator
    for data in generator_fn():

  File "D:\Programs\Anaconda\lib\site-packages\keras\engine\data_adapter.py", line 1050, in generator_fn
    yield x[i]

  File "D:\Programs\Anaconda\lib\site-packages\keras\preprocessing\image.py", line 116, in __getitem__
    ret

In [ ]:
print('No. of Samples:', train_generator.samples)
print('Number of Classes:', len(train_generator.class_indices))
print('Samples per Classes:', int(train_generator.samples/len(train_generator.class_indices)))

In [ ]:
# We get the third batch
train_generator.reset() #resets the generator to the first batch
for i in range(3):
    x1,y1 = next(train_generator)
    y1_int = np.argmax(y1,axis=-1)

#Plot the batch images w.r.t. the dataset images.
plt.figure(figsize=(4,4))
idx=1
for i in range(8):
    plt.subplot(4,4,idx)
    idx+=1
    plt.imshow(x1[i].reshape(128,128,3))
    plt.subplot(4,4,idx)
    # plt.axis('off')
    # label = y1[i]
    plt.imshow(io.imread(os.path.join(train_generator.directory,train_generator.filenames[(train_generator.batch_index-1)*32+i])))
    idx+=1

# plt.imshow(x1[1].numpy().astype('uint8'))
plt.savefig('./visual_original_comp.png', bbox_inches='tight')